In [9]:
import joblib
import shap
import pandas as pd

# Load Models

In [10]:
cost_model = joblib.load("saved_models/cost_model.pkl")
time_model = joblib.load("saved_models/time_model.pkl")
long_model = joblib.load("saved_models/long_session_model.pkl")

# Load Data

In [11]:
X_cost_train, X_cost_test, y_cost_train, y_cost_test = joblib.load("data/cost_train_test_split.pkl")
X_time_train, X_time_test, y_time_train, y_time_test = joblib.load("data/time_train_test_split.pkl")
X_long_train, X_long_test, y_long_train, y_long_test = joblib.load("data/long_train_test_split.pkl")

# Cost Shap

In [17]:
sample_cost = X_cost_test.iloc[[10]]

# create explainer using training background
explainer_cost = shap.Explainer(cost_model, X_cost_train)

# compute shap values for this instance
shap_values_cost = explainer_cost(sample_cost)

# convert to readable dataframe
df_cost_shap = pd.DataFrame({
    "Feature": sample_cost.columns,
    "SHAP Value": shap_values_cost.values[0],
    "Feature Value": sample_cost.values[0]
}).sort_values(by="SHAP Value", key=abs, ascending=False)

print(df_cost_shap)


                          Feature  SHAP Value  Feature Value
2                    Charger Type    5.397431       0.000000
0           Energy Consumed (kWh)    0.250973      41.248359
5                   Vehicle Model   -0.000259       3.000000
6              Charging Rate (kW)    0.000218      22.450837
1         State of Charge (End %)    0.000186      73.349959
3  Energy per 100 km (kWh/100 km)    0.000012      26.724671
4       Charging Station Location   -0.000004       4.000000


# TTime Shap

In [18]:
sample_time = X_time_test.iloc[[0]]

explainer_time = shap.Explainer(time_model, X_time_train)
shap_values_time = explainer_time(sample_time)

df_time_shap = pd.DataFrame({
    "Feature": sample_time.columns,
    "SHAP Value": shap_values_time.values[0],
    "Feature Value": sample_time.values[0]
}).sort_values(by="SHAP Value", key=abs, ascending=False)

print(df_time_shap)

                     Feature  SHAP Value  Feature Value
0               Long Session  -15.519395       1.000000
2  State of Charge (Start %)   -5.636690      76.218498
1     Battery Capacity (kWh)    5.325719      57.267087
3      Energy Consumed (kWh)    4.049982      93.997864
5        Charging Cost (USD)   -2.888868      12.219722
4              Vehicle Model   -2.014000       3.000000
6    State of Charge (End %)   -0.599816     100.000000


# Long Shap

In [19]:
sample_long = X_long_test.iloc[[0]]

# long is classification → shap.Explainer handles predict_proba automatically
explainer_long = shap.Explainer(long_model, X_long_train)
shap_values_long = explainer_long(sample_long)

# For classification we take class 1 (long session = yes)
class_index = 1 if shap_values_long.values.ndim == 3 else 0
values = shap_values_long.values[0][:, class_index] if shap_values_long.values.ndim == 3 else shap_values_long.values[0]

df_long_shap = pd.DataFrame({
    "Feature": sample_long.columns,
    "SHAP Value": values,
    "Feature Value": sample_long.values[0]
}).sort_values(by="SHAP Value", key=abs, ascending=False)

print(df_long_shap)

                              Feature  SHAP Value  Feature Value
1                  Charging Rate (kW)    0.193961      48.630961
2              Battery Capacity (kWh)    0.126321      57.267087
0  Charging Time Difference (minutes)   -0.071082     192.000000
5                    Temperature (°C)    0.036517      13.836768
4               Energy Consumed (kWh)    0.001469      93.997864
6           Charging Station Location    0.001379       3.000000
3                 Vehicle Age (years)    0.000769       6.000000
